**Missing Value Handling**

The original dataset contained many missing entries encoded as −200. These values were replaced with NaN and imputed using time-based interpolation, which preserves pollutant trends in environmental time series. Remaining gaps were filled with median values to avoid bias.

**Timestamp Construction**

The dataset includes separate Date and Time columns, which were merged into a unified Datetime index. This enabled chronological sorting and time-series feature extraction.

**Feature Scaling**

All numeric features were normalised using Min–Max scaling to ensure consistent learning behaviour across models and prevent features with larger magnitudes from dominating training.

**Temporal Feature Engineering**

We added hour, weekday, and month attributes to capture periodic patterns. Cyclic encodings (sine and cosine of hour) were included to represent continuous circular time transitions.

**Lag and Moving-Average Features**

To capture short-term temporal dependencies, pollutant columns were expanded with 1-hour, 3-hour, and 6-hour lag features. Long-term patterns were represented using 12-hour and 24-hour moving averages.

These engineered features significantly improve the capacity of machine learning models to forecast pollution levels.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Upload the file manually or place in /content
df = pd.read_csv("AirQualityUCI.csv", sep=';', decimal=',')
print(df.shape)
df.head()

# Remove unnamed column
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16'], errors='ignore')

# Replace −200 with NaN
df = df.replace(-200, np.nan)
df.isna().sum()

# Merge Date + Time making a Datetime index
# Explicitly specify the format to avoid parsing issues and drop rows where conversion fails
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'],
                                format='%d/%m/%Y %H.%M.%S', # Specify format
                                errors='coerce') # Coerce errors to NaT

# Drop original 'Date' and 'Time' columns
df = df.drop(columns=['Date', 'Time'])

# Drop rows where Datetime conversion resulted in NaT values
df.dropna(subset=['Datetime'], inplace=True)

df = df.sort_values('Datetime').reset_index(drop=True)
df = df.set_index('Datetime')


# Fix Data Types (ensure numeric)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.info()

# Interpolate time-based gaps
df = df.interpolate(method='time')

# Fill any remaining NaN with median
df = df.fillna(df.median())

# Feature Scaling
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df)
df_scaled = pd.DataFrame(scaled_array, columns=df.columns, index=df.index)

# Add temporal features
df_scaled['hour'] = df_scaled.index.hour
df_scaled['weekday'] = df_scaled.index.weekday
df_scaled['month'] = df_scaled.index.month

# Optional: cyclic encoding (better for ML models)
df_scaled['hour_sin'] = np.sin(2 * np.pi * df_scaled.index.hour / 24)
df_scaled['hour_cos'] = np.cos(2 * np.pi * df_scaled.index.hour / 24)

# Lag Features + Moving Averages
pollutants = ['CO(GT)', 'NOx(GT)', 'NO2(GT)', 'C6H6(GT)']

# Create lag features (1h, 3h, 6h)
for col in pollutants:
    df_scaled[f'{col}_lag1'] = df_scaled[col].shift(1)
    df_scaled[f'{col}_lag3'] = df_scaled[col].shift(3)
    df_scaled[f'{col}_lag6'] = df_scaled[col].shift(6)

# Create moving averages
for col in pollutants:
    df_scaled[f'{col}_ma12'] = df_scaled[col].rolling(window=12).mean()   # 12-hour MA
    df_scaled[f'{col}_ma24'] = df_scaled[col].rolling(window=24).mean()   # 24-hour MA

# Drop rows with NaN created by lags
df_scaled = df_scaled.dropna()

# Final data clean summary
df_scaled.info()
df_scaled.head()

# Export Clean Dataset
df_scaled.to_csv("air_quality_clean.csv")
print("Exported as air_quality_clean.csv")

(9471, 17)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9357 entries, 2004-03-10 18:00:00 to 2005-04-04 14:00:00
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CO(GT)         7674 non-null   float64
 1   PT08.S1(CO)    8991 non-null   float64
 2   NMHC(GT)       914 non-null    float64
 3   C6H6(GT)       8991 non-null   float64
 4   PT08.S2(NMHC)  8991 non-null   float64
 5   NOx(GT)        7718 non-null   float64
 6   PT08.S3(NOx)   8991 non-null   float64
 7   NO2(GT)        7715 non-null   float64
 8   PT08.S4(NO2)   8991 non-null   float64
 9   PT08.S5(O3)    8991 non-null   float64
 10  T              8991 non-null   float64
 11  RH             8991 non-null   float64
 12  AH             8991 non-null   float64
dtypes: float64(13)
memory usage: 1023.4 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9334 entries, 2004-03-11 17:00:00 to 2005-04-04 14:00:00
Data columns (total 38 columns):